<a href="https://colab.research.google.com/github/jwgdmkj/jupiterColab/blob/master/Boosting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""
지니계수 - 데이터 분포 균형 제대로 안 된거 판별 
ex) 가슴통증 있거나 없거나, 심장병 있는사람 없는사람
가슴통증 있고 심장병 있는사람 105, 없는사람 39
가슴틍증 없고 심장병있음 34 없음 125이면, 
1-(105/(105+39))^2-(39/(105+39))^2 가 곧 심장병있고 가슴통증없는 사람의 지니계수
지니가 낮을수록 분포균형이 잘 된 것

토탈 섬 가슴통증 있는 것의 지니게수는 0.395, 없는 건 0.336가 나오는데,
여기서 있는사람 총수 144와 없는사람 159이므로
(144*0.395)/(144+159) + (159*0.336)/(144+159)가 최종 0.364, 가슴통증의 총 지니계수가 됨

chest pain 잇는 사람중 심장병 있거나, 없거나등등을 세서, data를 생성
이를 통해, 어떤 feature가 더 좋은지 판별한다.
chest pain || good blood circulation || blocked arteries 등등과 심장병의 지니계수를 계산해,
낮을수록 더 순도가 높다는 것이며, 만일 good blood circulation과 심장병의 지니계수가 낮다면
심장혈관과 심장병의 상관관계가 제일 높단 것의 추측이 가능
이 가장 낮은 것을 맨위 Tree에 놓는다. 다음으론,

Good blood circulation을 제한 것과 심장병 지니관계중 제일 낮은걸 다음 node로 놓기

주의점! Tree를 무한정 길게 하지 마라. 어떤 노드의 새끼 노드의 지니계수가 더 높다면, 
부모보다 순도가 더 낮다는 것. 
하나의 노드로 나눠진 정보가 새끼보다 더 나아야 하는데, 새끼에서 나눴을 때 지니계수가 더 높다면
더 하는 의미가 없어진다.
부모는, 자식보다 그 가치가 좋아야 하는데, 새끼의 지니계수가 높다면 정보 가치가 떨어진다.

https://ratsgo.github.io/machine%20learning/2017/03/26/tree/

"""

from sklearn.datasets import load_iris
from sklearn import tree

In [2]:
iris = load_iris()

In [3]:
X, y= iris.data, iris.target
X.shape, y.shape

((150, 4), (150,))

In [4]:
X[0]

array([5.1, 3.5, 1.4, 0.2])

In [5]:
iris.feature_names
"""
꽃잎 길이와 너비, 꽃받침대 길이 너비에 관한 행
"""



'\n꽃잎 길이와 너비, 꽃받침대 길이 너비에 관한 행\n'

In [6]:
y[:100]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [7]:
"""
전체적으로 3개의 클래스 존재(꽃들의 종류) 0으로 나오면 setosa, 1번이면 versicolor,
2번이aus virginica
어떤 data가 input으로 들어올때 어떤 클래스에 해당하는 꽃인지 decision tree로 구해보자!
fully connected layer로도 가능할 테지만, 이 편도 가능.
"""
iris.target_names

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [8]:
model = tree.DecisionTreeClassifier()

In [9]:
model = model.fit(X,y) #인풋, 아웃풋 넣고 fit

In [10]:
import graphviz

#
dot_data = tree.export_graphviz(model, out_file = None)
graph = graphviz.Source(dot_data)

graph #트리 완성시키기

"""
지니계수는, 아래에서는 그냥 1에서 뺀 것이 아닌, 높은 게 더 좋다고 표시함
X[2]인 꽃잎 너비가 0.8보다 작느냐를 기준으로 트리를 구성  
"""

'\n지니계수는, 아래에서는 그냥 1에서 뺀 것이 아닌, 높은 게 더 좋다고 표시함\nX[2]인 꽃잎 너비가 0.8보다 작느냐를 기준으로 트리를 구성  \n'

In [11]:
iris.target_names

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [12]:
"""
만일 아래의 새 데이터가 들어왔다 할 때, 이 꽃이 어떤 품종인지 밝히고자 한다.
예측을 돌려본다. predict통해, decision tree에다가 X[50]을 넣어본다.
"""
X[50]

array([7. , 3.2, 4.7, 1.4])

In [13]:
"""
50번쨰 꽃은 첫 번째 클래스(종류의 꽃), 100번쨰 꽃은 두 번째 클래스(꽃)이다.
"""
model.predict([X[0], X[50], X[100]])

array([0, 1, 2])

In [14]:
"""
배깅 - 마치 투표처럼, decision 트리는 하나의 결과를 예측하는 것으로, 심장병이 있을지 없을지
판단하는데, 앙상블은 그 트리를 여러개 만들어, 예측 결과를 투표를 통해 의사결정 함.
(랜덤 포레스트)
이에 쓰이는 데이터는 일부만 쓰인다. 
의사결정이 완성된 모든 트리를 넣은뒤, 각각이 도출한 결과 중 가장 많은 결과치가 나온 걸로 결정
심장병 있는거 3개, 없는거 1개로 결과가 나왔다면 나온 것으로 하는 등....
"""
from sklearn.ensemble import RandomForestClassifier

#maxdepth = 
#샘플을 몇개로 쪼갤까 - mean sample
#estimate가 100 : 트리를 100개까지 만듬
rf_model = RandomForestClassifier(n_estimators= 10, max_depth = 4, random_state = 0)
rf_model = rf_model.fit(X,y)

In [15]:
rf_model.predict([X[0], X[1], X[2]])

array([0, 0, 0])

In [18]:
"""
배깅 - 연속적으로 sequential 하게 트리 생성해감. 
부스팅 - 각 트리를 평균내는 것이 아닌, 가중치도 계산, 곱해 weigthed sum 구함

첫번째 classifier 학습시키기 위해, 전채 training set 중 random sampling
에러로부터 모델의 weight를 계산
예측이 틀린 training data point의 weight 갱신, 다음 bootstrap에서 더 잘뽑히도록 함
일단 자른 트리의 정확도를 계산

각 트리는, 샘플링 부트스트랩 할 때마다 생성된다. 이 떄, 배깅은 서브샘플링을 다 랜덤으로 
독립적으로 하나, 여기선 한 번 트리를 학습시키고 전체 데이터셋에 예측 하면, 틀린 것에
가중치를 더 부여.다음 라운드에서 더 잘 뽑히도록 미세조정. 
이러면, 샘플을 뽑을 떄 이전의 틀렸던 데이터포인트가 포함될 확률 증가,
틀렸던 것을 반복적으로 트리로 학습시킴.
원래 데이터포인트 뽑힐 확률이 동일한데, 어떤 데이터포인트가 틀리면 이거의 확률을 늘리는 업뎃

배깅(랜덤포레스트) - 각 트리가 독립적이기에, 오버피팅 위험 적음, 안정적. 병렬적, 빠른 실행

부스팅(에이더부스트) - accuracy 높음. 지속적으로 학습해가며, 확률을 높이기위해 조정을 하기에, 
오버피팅.
"""

from sklearn.ensemble import AdaBoostClassifier
ab_model = AdaBoostClassifier(n_estimators = 10, random_state = 0)
ab_model = ab_model.fit(X,y)
ab_model.predict([X[0], X[1], X[2]])

array([0, 0, 0])